# Walk-Forward Backtest Report

This notebook loads the optional walk-forward report and visualizes per-horizon metrics.

In [ ]:
import sys, subprocess
from pathlib import Path

print('Python:', sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
subprocess.run(['pip', 'install', '-e', str(repo_root)], check=True)


In [ ]:
import json
from pathlib import Path

report_path = Path(repo_root) / 'reports' / 'walk_forward_report.json'
print('Report path:', report_path)
if not report_path.exists():
    raise FileNotFoundError('walk_forward_report.json not found. Run training with backtest enabled.')

report = json.loads(report_path.read_text())
list(report.get('targets', {}).keys())


In [ ]:
import pandas as pd
import plotly.express as px

# Example: plot per-horizon RMSE for load_mw
metrics = report['targets']['load_mw']['gbm']['per_horizon']
rows = pd.DataFrame([
    {'horizon': int(k), 'rmse': v['rmse'], 'mae': v['mae'], 'smape': v['smape']}
    for k, v in metrics.items()
]).sort_values('horizon')

fig = px.line(rows, x='horizon', y=['rmse','mae','smape'], title='Load MW - GBM per-horizon metrics')
fig


In [ ]:
# Compare models for a target (RMSE)

def plot_compare(target='load_mw'):
    out = []
    for model in ['gbm', 'lstm', 'tcn']:
        if model not in report['targets'][target]:
            continue
        per = report['targets'][target][model]['per_horizon']
        for k, v in per.items():
            out.append({'model': model, 'horizon': int(k), 'rmse': v['rmse']})
    df = pd.DataFrame(out).sort_values(['model','horizon'])
    fig = px.line(df, x='horizon', y='rmse', color='model', title=f'{target} - RMSE by horizon')
    return fig

plot_compare('load_mw')
